#### Ensemble 모형결합

1. Bagging   
같은 모형, 같은 데이터 샘플을 중복 사용하여 서로 다른 결과를 출력한 다수의 모형을 사용하는 방법
 * BaggingClassifier : 배깅 모형 결합을 위한 클래스
 * base_estimator : 기본 모형
 * n_estimators : 모형 갯수 (default 10)
 * bootstrap_features : 특징 차원의 중복 사용 여부 (default False)
 * max_features : 다차원 독립 변수 중 선택할 차원의 수 혹은 비율 1.0

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

model1 = DecisionTreeClassifier(random_state=0)
model2 = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, random_state=0)

for model in (model1,model2):
    print(model)
    model.fit(X_train, y_train) # train_test_split 했다고 가정
    print('학습용: ', model.score(X_train, y_train))
    print('검증용: ', model.score(X_test, y_test))
    print()

---
2. Random Forest   
- 의사결정나무를 개별 모형으로 사용하는 모형 결합 방법   
- 배깅은 사용하는 모형의 종류에 제한이 없으나, 랜덤포레스트는 의사결정나무 모형만을 사용
- 독립변수의 차원을 랜덤하게 감소시킨 후 독립변수를 선택하는 방법

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

model1 = DecisionTreeClassifier(random_state=0)
model2 = RandomForestClassifier(n_estimators=100, random_state=0)

for model in (model1,model2):
    print(model)
    model.fit(X_train, y_train) # train_test_split 했다고 가정
    print('학습용: ', model.score(X_train, y_train))
    print('검증용: ', model.score(X_test, y_test))
    print()

+) Extremely Randomized Trees
- 랜덤포레스트의 변정으로, Extra Trees라고도 한다
- 독립변수를 무작위로 선택하는 방식
- 랜덤포레스트는 DecisionTreeClassifier를 사용하지만, 엑스트라 트리는 ExtraTressClassifier(DecisionTreeClassifier를 상속한 클래스)를 사용  

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model3 = ExtraTreesClassifier(n_estimators=100, random_state=0)

for model in (model1,model2,model3):
    print(model)
    model.fit(X_train, y_train) # train_test_split 했다고 가정
    print('학습용: ', model.score(X_train, y_train))
    print('검증용: ', model.score(X_test, y_test))
    print()

> DecisionTree < RandomForest < ExtraTree 순으로 성능이 개선됨   
> 트리를 만드는 결정에 각 특성이 얼마나 중요한지를 평가하는 특성 중요도 적용   

---
3. Boosting   
- 미리 정해진 갯수의 모형을 사용하지 않고 하나의 모형에서 시작하여 개별 모형을 하나씩 추가하는 방법   
- 다수결 방법이 아닌 개별 모형의 출력에 가중치를 조합한 값을 판별함수로 사용

- base_estimetor 
    - 부스트 앙상블이 만들어지는 기본 분류 모형
    - 기본값 DecisionTreeClassifier (max_depth = 1)
- n_estimators 
    - 기본값 = 50
    - 부스팅이 종료되는 최대 추정량 (조기 종료)
- learning_rate 
    - 기본값 = 1
    - 학습진행 속도


 3-1. Adaboost    
- adaptive boost(적응형 부스트)
- 학습 데이터에 가중치를 주고 분류 모형이 틀리게 예측한 데이터의 가중치를 합한 값을 손실함수로 사용하고 손실함수를 최소화하는 모형을 선택하는 알고리즘







In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model_ada = AdaBoostClassifier(n_estimators=100, random_state=0)
model_ada.fit(X_train, y_train) # train_test_split 했다고 가정

- algorithm 
    - SAMME
    - SAMME.R(default = 'SAMME.R') : SAMME보다 빠르게 수렴되므로 부스팅 반복 횟수가 줄어 테스트 오류가 감소

In [ ]:
from sklearn.svm import SVC

svc = SVC(probability=True, kernel='linear')
model_svc = AdaBoostClassifier(algorithm='SAMME', n_estimators=50, base_estimator=svc)
model_svc.fit(X_train, y_train)

3-2. Gradientboost 
- 손실을 최소화하는 개별 분류함수를 찾는 알고리즘
- 내부적으로 의사결정 회귀나무 모형을 사용

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model_grad = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=0)

3-3. XGBoost
- 그레디언트 부스트 알고리즘을 분산환경에서도 실행할 수 있도록 구현된 라이브러리
- 성능이 우수하고 자원 활용률이 좋아서 최근 많이 사용되고 있는 알고리즘

In [ ]:
import xgboost

model_xgb = xgboost.XGBClassifier(n_estimators=100, max_depth=3, random_state=0)
%%time ##??
model_xgb.fit(X_train, y_train)

3-4. lightgbm
- GOSS(Gradient-based One_side Sampling)을 통해 데이터의 일부만으로 빠르게 정보이득을 계산하고 EFM(Exclusive Feature Bundling)을 통해 Feature를 획기적으로 감소시켜서 XGBoost보다 더 좋은 성능을 나타내는 알고리즘
- 정보이득(어떤 속성을 선택함으로 인해 데이터를 더 잘 구분하게 되는 특성)
    - 범주형 변수를 One Hot Incoding을 하게 되면 변수가 많이 늘어나서 계산이 오래 걸리는 상황이 발생하는데 lightgbm은 이 문제를 잝 극복할 수 있는 알고리즘